# Assignment 5
(v.2.1)
## Lecture 9 & 10 - Reinforcement Learning and Language Models

# General Instructions
- Jupyter notebook is supposed to be run cell by cell in order, please do not skip any code cell, this will cause some errors. Also running cells back and forth sometimes might also incur errors. If you feel you lost your track, you can click "Kernel->Restart" from the menu to restart the process.
- Before submitting your assignment, ensure that it does not contain trivial errors by pressing the "validate" button at the top.
- Your implementations are supposed to be added to the places where it reads "YOUR CODE HERE". Please also remove the "raise NotImplementedError()" line before submitting.
- Please DO NOT change the metadata of any cell, cells for demo and instructions are not editable.
- Please DO NOT change the order of solution cell and test cell, you will lost points if the order is changed.
- You can copy lines of code from cells that are not editable, but please DO NOT copy and paste them as cells, this may incur validation error. 
- You can add extra cells or code to help double-check your solution, but please make sure that variables required by tasks are not overwritten, or just delete those extra cells before submitting.
- Please DO NOT change file names in you submission, renamed files can not be recognized by the grading system.
- Reading the documentation of Python libraries is always a good practice, all the Python libraries (Numpy, Pandas, Sklearn,etc.) we utilized in this course provide very well organized documentation for each method/class/function.

# Reinforcement Learning

### Learning Goals
After successfully completing this assignment, you should be able to:
- explain why exploration and exploitation are important concepts in reinforcement learning problems
- understand better the Multi-Armed Bandit problem
- implement a strategy for maximizing the cumulative reward (UCB)
- understand the Frozen Lake game/problem
- implement an algorithm for training an agent to find an optimal path through this grid game (Q-learning)
- understand on an intuitive level how different parameters impacts the performance

## Table of Contents
1. [Multi-Armed Bandit](#multi_armed_bandit)
2. [Frozen Lake Game](#frozen_lake_game)

# Multi-Armed Bandit <a class="anchor" id="multi_armed_bandit"></a>

![Multi-armed bandit](https://leovan.me/images/cn/2020-05-16-multi-armed-bandit/compulsive-gambling.png "Multi-armed bandit")

Here we will focus on the "Multi-armed bandit" problem. There are 10 one-armed bandit machines where each have different probabilities for winning and with different reward ranges. As we have a limited amount of coins to spend, the goal is to play on these machines in a way that maximizes the cumulative reward. An action at every time step involves selecting and playing on  one of these ten one-armed bandits, or simply arms. For each action, i.e. for each arm that we play, we observe the reward which is a value between 0 and 10 (€). Since we know nothing about the reward properties of these machines to start with, we need to implement a suitable strategy that is based around observing the rewards that we get from playing on these machines.

In [1]:
# Import the libraries we will be using
import pandas as pd
import math
import random
random.seed(1234)
import numpy as np
import tabulate

## Dataset
The dataset that we will use here is `10_armed_bandit_data.csv`. Each row in this dataset shows the reward of playing on each arm at a given time step. With this way of representing the data, the reward from each arm changes for each time step, meaning that also the order in which the arms are selected will influence the outcome. Since there are 1000 rows in the dataset, it means we have 1000 coins to play with.

In [2]:
# Importing the dataset
df = pd.read_csv('10_armed_bandit_data.csv')

# Print the first 5 rows from the dataset.
print(df.head(5))

   Arm 0  Arm 1  Arm 2  Arm 3  Arm 4  Arm 5  Arm 6  Arm 7  Arm 8  Arm 9
0      0      0      0      4      2      0      0      0      0      1
1      0      0      6      0      0      0      0      0      0      0
2      0      0      0      0      3      0      0      0      0      0
3      0      0      7      0      2      0      1      5      0      0
4      0      2      0      5      0      0      0      0      0      0


## Random Selection
In this first part we adopt a simple strategy that involves naively selecting and playing on a random arm each time, independent of the rewards we get.

<div class=" alert alert-warning">
    
## Student Task A5.1

Your task is to implement the Random Selection strategy - selecting a random arm (or action) $a$ at every time step $t$ - by completing the `random_selection()` function. Our budget is 1000 coins.
</div>

In [22]:
# Implementing Random Selection 
def random_selection(data):
    
    # There are 10 columns in the dataset, one for each arm/machine.
    # The row count represents the number of times we play, i.e., how many coins we will spend. 
    coins, arms = data.shape  # (1000, 10)
    
    reward_sum_per_arm = [0] * arms
    selection_count_per_arm = [0] * arms
    
    for t in range(0, coins):
        
        ## Select a random arm (action) among the 10 available arms
        # a = random. ...
        
        ## Fetch the reward for selecting the given arm (a) at the given time step (t) from the data
        # reward = data.values[...]
        
        # YOUR CODE HERE
        #raise NotImplementedError()
        a = random.randint(0, arms - 1)
        reward = data.values[t,a]

        selection_count_per_arm[a] += 1
        reward_sum_per_arm[a] += reward

    return reward_sum_per_arm, selection_count_per_arm


# Run the selection algorithm
reward_sum_per_arm, selection_count_per_arm = random_selection(df)

# Print the outcome and some statistics
print('Total reward using Random Selection: {}'.format(sum(reward_sum_per_arm)))

sorted_indxs = np.argsort([-x for x in selection_count_per_arm])
tab_data = [[indx, selection_count_per_arm[indx]] for indx in sorted_indxs]
headers = ['Arm', 'Times selected']
print('\n' + tabulate.tabulate(tab_data, headers=headers))

Total reward using Random Selection: 754

  Arm    Times selected
-----  ----------------
    9               120
    8               111
    6               107
    2               104
    7                99
    5                95
    0                94
    1                91
    4                90
    3                89


In [23]:
# this cell is for tests


## Upper Confidence Bound (UCB) Selection

Random selection of arms to play on is probably not the optimal approach. To improve our reward we want to use the UCB algorithm to calculate $a_t$ which is the action, or arm, at time $t$ with maximum UCB:
$$a_t = \underset{a\in{A}}{\operatorname{argmax}}\biggr[Q_t(a) + c \sqrt{\frac{\log{t}}{N_t(a)}}\biggr]$$
<br>
Where $Q_t(a)$ is the estimated value of action (arm) $a$ at time step $t$:
$$Q_t(a) = \frac{ \sum_{i=1}^{t-1} R_{i}\mathbf{1}_{A_i = a} }{ \sum_{i=1}^{t-1} \mathbf{1}_{A_i = a} }$$
<br>
$t$ - current time step.<br>
$N_t(a)$ - the number of times that action $a$ has been selected, prior to time $t$.<br>
$c$ - confidence value that controls the level of *exploration* vs *exploitation*. Higher $c$ means higher exploration rate (... of arms that have not been explored much or yielded much reward so far).<br>
$\sum_{i=1}^{t-1} R_{i}\mathbf{1}_{A_i = a}$ - sum of rewards from selecting $a$ up to $t$-1<br>
$\sum_{i=1}^{t-1} \mathbf{1}_{A_i = a}$ - number of times $a$ has been selected up to $t$-1<br>


<div class=" alert alert-warning">
    
## Student Task A5.2

In this assignment your task is to implement the above UCB algorithm, `max_ucb_action()`. As inputs our function takes $c$, $t$, a list storing the accumulated rewards per arm (`reward_sum_per_arm`), and a list with informaion about how many times each arm has been selected so far (`selection_count_per_arm`, this is $\{N_t(A)\}$).
<br><br>
Note, in the beginning the value of $N_t(a)$ will be zero. To avoid the division by zero problem, we can, in those cases, set it to a number that is close to zero, e.g., 1e-100.
</div>

In [24]:
# Example showing how the input arguments might look like at t = 40.
# c = 1
# t = 40
# reward_sum_per_arm = [0, 12, 0, 2, 1, 0, 0, 0, 0, 0]
# selection_count_per_arm = [3, 15, 3, 5, 4, 3, 2, 2, 2, 2]

# reward_sum_per_arm and selection_count_per_arm stores the cummulated values up to the current time step t

def max_ucb_action(c, t, reward_sum_per_arm, selection_count_per_arm):
    arms = len(reward_sum_per_arm)
    confidence_per_arm = [0] * arms
    
    # In this for-loop we calculate the confidence_per_arm (i.e., action a) at the given time step t
    for a in range(0, arms):
        
        Nt = selection_count_per_arm[a]
        if Nt == 0:
            Nt = 1e-100  # To avoid division by zero error
        
        # Qt = ...
        
        ## NB!: use math.log(t+1) to avoid error with math.log(0)
        # Ut = ...  
        
        # confidence_per_arm[a] = ...
        
        
        # YOUR CODE HERE
        #raise NotImplementedError()
        Qt = reward_sum_per_arm[a] / Nt
        Ut = c * math.sqrt(math.log(t+1) / Nt)
        confidence_per_arm[a] = Qt + Ut
        

    # Pick the arm/action with the highest upper bound
    max_upper_bound_arm = confidence_per_arm.index(max(confidence_per_arm))
    return max_upper_bound_arm

In [25]:
# this cell is for tests


<div class=" alert alert-warning">
    
## Student Task A5.3

Now that we have implemented `max_ucb_action()`, the next thing to do is to implement the function `ucb_selection()` where we use the UCB selection approach for playing on these 10 one-armed bandits. Our budget is again 1000 coins.
</div>

In [29]:
# Implementing UCB Selection
def ucb_selection(data, c):
    
    ## Implement this in the same way as the above random_selection() function (from A5.1).
    ## The only exception is that you should now use use max_ucb_action() to select arms 
    ## instead of selecting arms at random.
    # ...
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    coins, arms = data.shape  # (1000, 10)
    reward_sum_per_arm = [0] * arms
    selection_count_per_arm = [0] * arms
    #return reward_sum_per_arm, selection_count_per_arm


    
    # There are 10 columns in the dataset, one for each arm/machine.
    # The row count represents the number of times we play, i.e., how many coins we will spend. 
    coins, arms = data.shape  # (1000, 10)
    
    reward_sum_per_arm = [0] * arms
    selection_count_per_arm = [0] * arms
    
    for t in range(0, coins):
        
        ## Select a random arm (action) among the 10 available arms
        # a = random. ...
        
        ## Fetch the reward for selecting the given arm (a) at the given time step (t) from the data
        # reward = data.values[...]
        
        # YOUR CODE HERE
        #raise NotImplementedError()
        a = max_ucb_action(c, t, reward_sum_per_arm, selection_count_per_arm)
        reward = data.values[t,a]
        
        selection_count_per_arm[a] += 1
        reward_sum_per_arm[a] += reward

    return reward_sum_per_arm, selection_count_per_arm
    

# Run the selection algorithm (by default we have set c = 1)
c = 1
reward_sum_per_arm, selection_count_per_arm = ucb_selection(df, c)

# Sanity check
assert(sum(ucb_selection(df, c=1)[0]) == 1200)


# Print the outcome and some statistics
print('Total reward using UCB Selection:', sum(reward_sum_per_arm))

sorted_indxs = np.argsort([-x for x in selection_count_per_arm])
tab_data = [[indx, selection_count_per_arm[indx]] for indx in sorted_indxs]
headers = ['Arm', 'Times selected']
print('\n' + tabulate.tabulate(tab_data, headers=headers))

Total reward using UCB Selection: 1200

  Arm    Times selected
-----  ----------------
    3               874
    9                61
    7                35
    4                 6
    0                 4
    1                 4
    2                 4
    5                 4
    6                 4
    8                 4


In [30]:
# this cell is for tests


# The Frozen Lake Game <a class="anchor" id="frozen_lake_game"></a>
<img src="https://debuggirl.files.wordpress.com/2013/03/img_43081.jpg" alt="drawing" width="800"/>

The Frozen Lake game is about safely navigating a frozen lake whose ice happen have several hidden cracks or holes in it, stepping on any of these will result in you falling into the water. The goal is to reach a part of the lake where a hidden treasure is located. Both the weak spots (holes) and the goal are not known beforehand by the player. Here we will train an *agent* to navigate this lake safely and reach the goal. To do so we will be using the **Q-learning** algorithm. Here we use the **Q-value function** $Q({s, a}) = v$ that takes as input the current state $s$ and possible action $a$, and returns its value $v$. The frozen lake is represented as a grid of cells (gridworld), where each cell is associated with a state that can be start `S`, frozen lake `F`, hole in the ice `H`, or goal `G`, and the actions available at each state is to navigate *left*, *down*, *right*, or *up*.<br>
<br>
When playing this game the agent will be iteratively doing the following:
1. Choose an action given the current state of the agent (calculate where to move).
2. Perform the chosen action (move to the chosen cell).
3. Upadete the Q-value function (the Q-table) based on the reward from performing this action.
4. Set the new state of the agent.

In [31]:
# Import the libraries we will be using
import gym  # gym-0.21.0
#gymnasium as gym
import numpy as np
import time
import random
from IPython.display import clear_output
import os

print(gym.__version__)

# Initiate the FrozenLake game
env = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")

# Next we render the game board
env.render()

0.21.0

SFFF
FHFH
FFFH
HFFG


After running the above, you should should see the gameboard, i.e. the frozen lake.
<br>
SFFF</br>
FHFH<br>
FFFH<br>
HFFG<br>


It consist of the following cells:
 - S - Start.
 - F - Frozen lake, which is safe to walk on.
 - H - Hole in the ice. Reaching this cell ends the round.
 - G - Goal. Reaching this cell gives +1 point and ends the round.

As mentioned, we will now train an agent to navigate this frozen lake and ultimately reach the goal (`G` cell) with as few steps/turns as possible without falling into the lake (stepping on any of the `H` cells). At start the agent does not know where the goal is or what cells are safe to walk on. However, with repeated trials, combined with the delayed feedback reward gained from stepping on the various cells, the agent should eventually learn to navigate this lake.

In [32]:
state_space_size = env.observation_space.n
action_space_size = env.action_space.n
print('The frozen lake (gameboard) consist of {} states (cells) and {} actions for each of these.'.format(state_space_size, action_space_size))
print('So for each state there are four actions availble:\n0 = ◀️ left\n1 = 🔽 down\n2 = ▶️ right\n3 = 🔼 up')

The frozen lake (gameboard) consist of 16 states (cells) and 4 actions for each of these.
So for each state there are four actions availble:
0 = ◀️ left
1 = 🔽 down
2 = ▶️ right
3 = 🔼 up


## Q-Table
We will be using a **Q-table** as the underlying representation of the value function (Q-value function). For this type of game board, the Q-table is a well suited representation to use. This table will store all the knowledge gained by the agent - each *state* will have information about the *value* of taking any of the available *actions* (◀️, 🔽, ▶️, 🔼) when the policy is to find the hidden treasure without falling into the lake. To start with, all the q-values in this table will be zero, but these will be updated as the agent is playing.

In [33]:
q_table = np.zeros((state_space_size, action_space_size))

# Print some info about it
s = 'Q-table:\n'
i = 0
j = 0
for cr in q_table:
    s += '{}\t<-- state [{},{}]\n'.format(cr, i, j)
    j += 1
    if j >= action_space_size:
        j = 0
        i += 1
print(s)

Q-table:
[0. 0. 0. 0.]	<-- state [0,0]
[0. 0. 0. 0.]	<-- state [0,1]
[0. 0. 0. 0.]	<-- state [0,2]
[0. 0. 0. 0.]	<-- state [0,3]
[0. 0. 0. 0.]	<-- state [1,0]
[0. 0. 0. 0.]	<-- state [1,1]
[0. 0. 0. 0.]	<-- state [1,2]
[0. 0. 0. 0.]	<-- state [1,3]
[0. 0. 0. 0.]	<-- state [2,0]
[0. 0. 0. 0.]	<-- state [2,1]
[0. 0. 0. 0.]	<-- state [2,2]
[0. 0. 0. 0.]	<-- state [2,3]
[0. 0. 0. 0.]	<-- state [3,0]
[0. 0. 0. 0.]	<-- state [3,1]
[0. 0. 0. 0.]	<-- state [3,2]
[0. 0. 0. 0.]	<-- state [3,3]



With an empty Q-table the agent do not know anything about what actions are better than others. In other words, the Q-value function will always return zero for every possible action at each state. Thus in the beginning the agent will instead be taking random actions while exploring the board. This is called *exploration*. However, after having gained some knowledge about the board, the agent can start relying on the experience gained and reflected in the Q-value function. This is called *exploitation*. We will be using the *epsilon greedy strategy* to find a balance between *exploration* (random actions) and *exploitation* (optional actions according to the Q-value function).

In [34]:
# Here we introduce some parameters to use in the Q-training algorithm.

# Learning rate (value between 0 and 1)
learning_rate = 0.1

# Reward discount used in the Bellman equation (value between 0 and 1)
discount_factor = 0.99

## The Bellman Equation
For updating a state in the Q-table we will here be using the following version of the *Bellman equation*:<br>
$$Q^{updated}(s_{t}, a_{t}) := (1-\alpha)Q(s_{t}, a_{t}) + \alpha (r_{t} + \gamma \underset{a}{\max}Q(s_{t+1},a))$$
$s_{t}$ - current state.<br>
$a_{t}$ - chosen action.<br>
$\alpha$ - learning rate.<br>
$r_{t}$ - reward from taking the action $a_{t}$.<br>
$\gamma$ - discount factor.<br>
$s_{t+1}$ - new state after taking action $a_{t}$.<br>
$\underset{a}{\max}Q(s_{t+1},a)$ - the optional value possible from $s_{t+1}$.<br>

<div class=" alert alert-warning">
    
## Student Task A5.4

<strong><i><u>Your task<i></u></strong> is to implement the Bellman equation in `q_update()` that calculates and returns the new value for $Q^{updated}(s_{t}, a_{t})$.<br>It takes as input: Q-table, $s_{t}$, $a_{t}$, $s_{t+1}$, $r_{t}$, $\alpha$, and $\gamma$.
</div>

In [35]:
def q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor):
    # Calculate the new value in the q_table using Bellman equation

    ## Implement the Bellman equation as shown above
    # q_new = q_table[state, action] * ...
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    q_new = q_table[state, action] * (1-learning_rate) + learning_rate * ( reward + discount_factor * max(q_table[new_state]))
    
    return q_new

In [36]:
# this cell is for tests


In [37]:
# For the mentioned epsilon greedy strategy we will need a few more parameters.

# Upper bound for the exploration rate (value between 0 and 1)
max_exploration_rate = 1.0

# Lower bound for the exploration rate (value between 0 and 1)
min_exploration_rate = 0.01

# How much the exploration rate decays per episode (down to the min_exploration_rate, value between 0 and 1).
exploration_decay_rate = 0.001

## Epsilon Greedy Strategy
By using the *epsilon greedy strategy*, we will let the exploration rate start high (1), and then decrease for every episode with exponential decay. The goal here is to find the optimal balance between exploration and exploitation. Each round will consist of 10000 episodes. In the first episodes we want to have a high probility of performing exploration actions to avoid getting stuck in sub-optimal paths (local maximums). However, we want to gradually start leaning more towards exploitation towards the later episodes.<br>
<br>
We will be using the following formula for calculating the exploration rate for a given episode $\epsilon(i)$:<br>
$$\epsilon(i) = \epsilon_{min} + (\epsilon_{max} - \epsilon_{min}) \cdot e^{-\zeta \cdot i}$$
$\epsilon$ - exploration rate.<br>
$i$ - episode number.<br>
$\epsilon_{min}$ - min exploration rate.<br>
$\epsilon_{max}$ - max exploration rate.<br>
$\zeta$ - exploration decay rate.<br>

<div class=" alert alert-warning">
    
## Student Task A5.5

Implement the `calc_exploration_rate()` function which calculates and returns the exploration rate using the above formula.
</div>

In [39]:
# Function that calculate the exploration rate for a given episode
def calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate):
    ## Calculate the exploration rate
    # exploration_rate = ...
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) *  math.exp((-exploration_decay_rate * episode))
    
    return exploration_rate

In [40]:
# this cell is for tests


<div class=" alert alert-warning">
    
## Student Task A5.6

Next we will make the function for choosing an action given a state. This function includes an *exploration rate threshold* which is a randomly drawn number between 0 and 1. For every action, this thershold makes the agent perform an exploration action with a probability equal to the given *exploration rate*.
<br>
<strong><i><u>Your task<i></u></strong> is to complete the below function `choose_action()`. This function takes as input the Q-table, current state $s_{t}$, and exploration rate $\epsilon$. It should return an integer reflecting the chosen action (from the set {0, 1, 2, 3}).
</div>

In [42]:
def choose_action(q_table, state, exploration_rate):
    
    ## 1. Pick a random number between 0 and 1
    # exploration_rate_threshold = ...
    
    exploration_rate_threshold = np.random.uniform(0, 1)
    ## 2. Exploration vs exploitation
    if exploration_rate > exploration_rate_threshold or sum(q_table[state, :]) == 0:
        ## Exploration, i.e., random action 
        # action = ...
        #pass
        action = np.random.randint(q_table.shape[1])
    else:
        ## Exploitation, i.e., perform the optimal action at this state according to the Q-table
        # action = ...
        #pass
        action = np.argmax(q_table[state, :])
    
    
    # YOUR CODE HERE
    #raise NotImplementedError()

    return action

In [43]:
# this cell is for tests


## Q-Learning Algorithm
Now it is time to train our agent using the Q-learning algorithm as shown below.
If everything works as intended, the below code should print some statistics from the training.

In [44]:
# Number of episodes to train
num_episodes = 10000

# Max number of steps before ending the round
max_steps_per_episode = 100

In [45]:
# Q-learning function
def train_q_learn(env, q_table):
    
    # Q-Learning algorithm
    rewards_all_episodes = []
    steps_all_episodes = []
    exploration_rate = max_exploration_rate

    # Play the game for n episodes
    for episode in range(num_episodes):
        state = env.reset()
        done = False
        rewards_current_episode = 0
        steps_current_episode = 0

        for step in range(max_steps_per_episode):
            # Chooce an action. Exploration vs exploitation trade-off with the epsilon greedy strategy
            action = choose_action(q_table, state, exploration_rate)

            # Perform the chosen action
            new_state, reward, done, info = env.step(action)

            # Upadete the Q-table
            q_table[state, action] = q_update(q_table, state, action, new_state, reward, learning_rate, discount_factor)

            # Set the new state
            state = new_state

            # Collect some statistics
            rewards_current_episode += reward
            steps_current_episode += 1

            if done == True:
                break

        # Update the exploration rate
        exploration_rate = calc_exploration_rate(episode, min_exploration_rate, max_exploration_rate, exploration_decay_rate)

        rewards_all_episodes.append(rewards_current_episode)
        steps_all_episodes.append(steps_current_episode)


    # Print the average rewards
    rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
    steps_per_thousand_episodes = np.split(np.array(steps_all_episodes), num_episodes/1000)

    print('Training results, average reward and steps per thousand episodes:')
    count = 1000
    for i, _ in enumerate(rewards_per_thousand_episodes):
        r = rewards_per_thousand_episodes[i]
        s = steps_per_thousand_episodes[i]
        print('{}: rewards {:.4}, steps {:.4}'.format(count, sum(r/1000), sum(s/1000)))
        count += 1000
    return q_table

In [46]:
# Run the train_q_learn function
q_table = train_q_learn(env, q_table)

Training results, average reward and steps per thousand episodes:
1000: rewards 0.277, steps 8.179
2000: rewards 0.73, steps 6.839
3000: rewards 0.895, steps 6.327
4000: rewards 0.969, steps 6.181
5000: rewards 0.985, steps 6.089
6000: rewards 0.99, steps 6.05
7000: rewards 0.982, steps 6.009
8000: rewards 0.989, steps 6.029
9000: rewards 0.991, steps 6.042
10000: rewards 0.987, steps 6.04


In [47]:
print('Trained Q-table:')
print(q_table)

Trained Q-table:
[[0.94148015 0.93206535 0.95099005 0.94148015]
 [0.94148015 0.         0.96059601 0.95099005]
 [0.95099005 0.970299   0.95099003 0.96059601]
 [0.96059601 0.         0.87529013 0.82383882]
 [0.91821032 0.78112473 0.         0.94148015]
 [0.         0.         0.         0.        ]
 [0.         0.9801     0.         0.96059601]
 [0.         0.         0.         0.        ]
 [0.19309012 0.         0.94016893 0.57099703]
 [0.69907888 0.89308457 0.98009987 0.        ]
 [0.97029643 0.99       0.         0.970299  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.68000273 0.98999996 0.74526666]
 [0.98009929 0.98999991 1.         0.98009999]
 [0.         0.         0.         0.        ]]


## Playing Frozen Lake
Finally we will let the agent play Frozen Lake using the Q-table that we have learned above.

<div class=" alert alert-warning">
    
## Student Task A5.7

Now we are ready to put the agent to navigate the frozen lake based on what it has learned. By simply looking at the Q-table, maybe you can already see what route the agent will be taking?<br><br>
<strong><i><u>Your task<i></u></strong> is to run the below code and observe how the agent is navigating the frozen lake. Observe the learned Q-table to understand why the agent is taking this specific path.
</div>

In [48]:
def run_check():
    nbgrader_exec_env = os.environ.get('NBGRADER_EXECUTION')
    if nbgrader_exec_env is not None and nbgrader_exec_env in ['autograde', 'validate']:
        return False
    return True

# Defining the run_agent function
def run_agent(env, q_table):
    env.reset()
    env.render()
    if run_check():
        clear_output(wait=False)
        # Watch the agent play for a few rounds using the learned Q-table
        n_rounds = 3
        for episode in range(n_rounds):
            state = env.reset()
            done = False

            print('༺❁ EPISODE {} ❁༻'.format(episode+1))
            time.sleep(1)

            for step in range(max_steps_per_episode + 1):
                clear_output(wait=True)
                env.render()
                time.sleep(0.3)

                action = np.argmax(q_table[state, :])
                
                new_state, reward, done, info = env.step(action)

                if done:
                    clear_output(wait=True)
                    env.render()
                    if reward == 1:
                        print('༺❁ You reached the goal after {} steps! ❁༻'.format(step + 1))
                        time.sleep(3)
                    elif step == max_steps_per_episode - 1:
                        print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
                        time.sleep(3)
                    else:
                        print('༺❁ You fell through a hole after {} steps! ❁༻'.format(step + 1))
                        time.sleep(3)
                    clear_output(wait=True)
                    break
                elif step == max_steps_per_episode - 1:
                    print('༺❁ No steps left ({} steps max)! ❁༻'.format(step + 1))
                    time.sleep(3)
                    clear_output(wait=True)
                    break
                state = new_state
        time.sleep(3)
        print('༺❁ Done ❁༻')
        env.close()

In [49]:
# Run the run_agent function with the trained Q-table
run_agent(env, q_table)

༺❁ Done ❁༻


## The Ice Can Get Slippery!
![Slippery ice](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS7iaMa5OcNHtJFKB1fZo_EvG-nQyPPTm_fsA&usqp=CAU "Slippery ice")

So far the ice has been easy to walk on, and we have seen that the agent has not had any problem in finding an optimal path leading to the goal. Now, however, the weather has changed and the ice has suddenly become very slippery.
In the next part of this exercise we have defined the frozen lake as:<br>
`env_frozen = gym.make('FrozenLake-v1', is_slippery=False, map_name="4x4")`<br>
`is_slippery=True` means that there is only a 1/3 chance that the agent will move in the intended 
direction, and 2/3 chance to move in one of the perpendicular directions (1/3 and 1/3). 
E.g., if **action = left**, then:
- P(move left) = 1/3
- P(move up) = 1/3
- P(move down) = 1/3

<div class=" alert alert-warning">
    
## Student Task A5.8a

First run the Q-Learning algorithm below and observe how the new trained Q-table looks like, and how the new statistics has become. Also observe how the behaviour of the agent has changed at run time.
    
## Student Task A5.8b
Next we want you to briefly explore the impact of different values assigned to the different parameters, and try to find the most promising values that optimize the reward gained by the agent (look at the scores from the last 1000 episodes).
The parameters in question are:
- learning_rate ($\alpha$)
- discount_factor ($\gamma$)
- max_exploration_rate ($\epsilon_{max}$)
- min_exploration_rate ($\epsilon_{min}$)
- exploration_decay_rate ($\zeta$)

<u>Be prepared to discuss your findings with your colleges and/or teaching assistants.</u><br>
Some additional things to think about: Why is the preferred first move of the agent to move left? Can you think of some possible approaches and strategies that could improve the performance of the agent further? Can you think of some completely different ways for how to solve this game/problem?
</div>

In [79]:
# Initiate the FrozenLake game with slippery ice
env_slippery = gym.make('FrozenLake-v1', is_slippery=True, map_name="4x4")


## Parameters used by the Q-training algorithm

# Learning rate (value between 0 and 1)
learning_rate = 0.9

# Reward discount used in the Bellman equation (value between 0 and 1)
discount_factor = 0.99

# Upper bound for the exploration rate (value between 0 and 1)
max_exploration_rate = 1.0

# Lower bound for the exploration rate (value between 0 and 1)
min_exploration_rate = 0.01

# How much the exploration rate decays per episode (down to the min_exploration_rate, value between 0 and 1).
exploration_decay_rate = 0.001


# Train the agent to navigate the slippery ice
q_table_slippery = np.zeros((env.observation_space.n, env.action_space.n))
if run_check():
    q_table_slippery = train_q_learn(env_slippery, q_table_slippery)

Training results, average reward and steps per thousand episodes:
1000: rewards 0.036, steps 9.404
2000: rewards 0.062, steps 15.05
3000: rewards 0.161, steps 23.81
4000: rewards 0.277, steps 33.19
5000: rewards 0.357, steps 40.01
6000: rewards 0.35, steps 44.35
7000: rewards 0.475, steps 45.03
8000: rewards 0.442, steps 49.46
9000: rewards 0.425, steps 47.13
10000: rewards 0.411, steps 46.85


In [80]:
print('Trained slippery Q-table:')
print(q_table_slippery)

Trained slippery Q-table:
[[6.02102441e-01 1.51020564e-01 9.33706991e-02 1.48461009e-01]
 [3.44449213e-02 8.52076365e-03 1.52074997e-02 1.62132154e-01]
 [1.37940227e-02 2.76324119e-02 1.20009883e-01 1.48538845e-01]
 [3.12766359e-02 1.54643787e-03 1.25250292e-01 1.29050974e-01]
 [7.22682240e-01 6.49625501e-02 7.16509380e-02 1.98007704e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.00809914e-04 1.17153511e-08 1.66491247e-04 1.39185040e-06]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [4.41121921e-02 5.28434078e-02 7.18899792e-02 8.83517810e-01]
 [5.68898803e-02 9.59118810e-01 1.78780760e-02 3.23872458e-02]
 [9.52214295e-01 1.16674696e-04 2.09394970e-04 1.12139712e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [7.14764863e-02 7.96824036e-03 9.62178144e-01 6.49373082e-02]
 [1.77832033e-01 9.97844887e-01 1.08486215e-01 9.43707729e-02]
 [0.00000000e+00 0.00000000e+

In [81]:
# Run the agent to see how the agent now navigates the slippery ice.
run_agent(env_slippery, q_table_slippery)

༺❁ Done ❁༻


# Language Models
<img src="https://www.wisecube.ai/wp-content/uploads/2023/05/Featured-Blog-Image-A-Comprehensive-Overview-of-Large-Language-Models-1080x675.jpg" alt="drawing" width="800"/>

Language models, or large language models (LLMs), primarily focuses on classifying and generating text (next token prediction), as well as generating condensed vector representations of texts, typically referred to as embeddings. 
Pre-training on large datasets, in conjunction with task-specific supervised fine-tuning, demonstrates state of the art performance in many natural language processing (NLP) tasks, including few-shot and zero-shot scenarios. In this assignment we will explore the application of LLM in completing two tasks.

The transformer library by Huggingface will be used for ease of implementation. It is built for simplifying the implementation of NLP applications. Huggingface is also known for their model and dataset hub, where they host various trained models and datasets which allows users to easily exchange trained models and datasets, as well as showcase demos of their models.


### Learning Goals
After successfully completing this part of the assignment, you should be able to:
- solve NLP tasks using existing LLM in a zero-shot manner.
- learn how to use existing LLMs to generate texts.
- understand the basics of how to evaluate generative LLMs.
- understand how to use huggingface library and Huggingface model hub.
- implement similarity search for doing query-based document retrieval.

## Table of Contents
1. [Language Generation](#language_gneration)
2. [Zero-Shot Information Retrieval](#zero_shot_information_retrieval)

# Language Generation <a class="anchor" id="language_gneration"></a>

The task involves generating textual content in response to a given prompt. 
It serves as the foundation for numerous natural language generation (NLG) tasks. 
In this work, the GPT-Neo Model with 68 Million parameters will be utilized. 
Despite its very modest size, the model has been pre-trained on the TinyStories dataset, enabling it to generate plausible sentences.

In [78]:
# Import the libraries we will be using
import math
import os
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer,logging
logging.set_verbosity(logging.ERROR )
import torch
import warnings
warnings.filterwarnings("ignore")

# Helper function for grading
def run_check():
    nbgrader_exec_env = os.environ.get('NBGRADER_EXECUTION')
    if nbgrader_exec_env is not None and nbgrader_exec_env in ['autograde', 'validate']:
        return False
    return True

# Model checkpoint name from Huggingface hub
model_name = 'roneneldan/TinyStories-33M'

# Load the pre-trained model
# AutoModelForCausalLM is for language model tasks
model = AutoModelForCausalLM.from_pretrained(model_name)

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Example prompt for text generation
prompt = "Hi, How are you?"

# Tokenize the prompt
# return_tensors="pt" specifies PyTorch tensors (use "tf" for TensorFlow)
input_ids = tokenizer.encode(prompt, return_tensors="pt")

# Generates up to max_length words from the prompt.
# top_p, num_beams, etc., control generation (details: https://huggingface.co/transformers/v4.2.2/_modules/transformers/generation_utils.html).
# return_dict_in_generate and output_scores: return prediction scores along with tokens.
# pad_token_id:ID of the padding token.
outputs = model.generate(input_ids, max_length=200, return_dict_in_generate=True, output_scores=True, pad_token_id=50256)

# Decode the generated tokens back to text
# skip_special_tokens=True: special tokens such as [CLS], [SEP] which we skip in the output text.
output_text = tokenizer.decode(*outputs[0], skip_special_tokens=True)

print(f'Given the prompt: "{prompt}"\nThe generated text corresponding to this prompt is: \n\n {output_text} \n\n\n ')

# outputs[1] contains the logit scores for each token. 
# Typically, these logits are converted to probabilities using the Softmax function.
probabilities = [max(torch.nn.functional.softmax(x)[0].tolist()) for x in outputs[1]]


Given the prompt: "Hi, How are you?"
The generated text corresponding to this prompt is: 

 Hi, How are you? Do you want to play with me?"

The boy looked at her and said, "No, I don't. Go away. You are too small and silly. You don't know how to play. You are a baby."

Lily felt sad and angry. She said, "No, you are not. You are mean and rude. You don't know how to be nice. You don't know how to have fun. You don't know how to be a friend."

The boy laughed and said, "You are just a baby. You don't know how to have fun. You don't know how to be happy. You don't know how to be a friend. You are a bully."

Lily felt hurt and scared. She said, "No, I am not. I am a friend. I am a bully. I don't know how to be nice. I don't know how to be 


 


<div class=" alert alert-warning">
    
## Student Task A5.9

Your task is to implement the `text_generation()` function, whose stages are already demonstrated in the cell above. The function will be invoked for three prompts with respective max_len values of 50, 100, and 60.
</div>

In [99]:
# Implementing Text Generation
def text_generation(model,tokenizer,prompt,max_len):
    ''' 
    Given a prompt and model, generate text of length max_len.
    
    Args:
    model: The pre-trained language model
    tokenizer: The tokenizer associated with the model
    prompt (str): The input text to generate from
    max_len (int): The maximum length of the generated text
    
    Returns:
    tuple: (generated_text, token_probabilities)
    
    ''' 
    
    # input_ids = tokenizer.
    # outputs = model.generate()
    # output_text = 
    # probabilities = 
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    input_ids = tokenizer.encode(prompt, return_tensors ="pt")
    outputs = model.generate(input_ids, max_length=max_len, return_dict_in_generate=True, output_scores=True)
    output_text =tokenizer.decode(*outputs[0], skip_special_tokens=True)
    probabilities = [max(torch.nn.functional.softmax(x)[0].tolist()) for x in outputs[1]]
    
    return output_text,probabilities

prompt = "once upon a time "
text,prob = text_generation(model,tokenizer,prompt,max_len=50)
print(text)

once upon a time  there was a little girl named Lucy. She was three years old and loved to explore. One day, Lucy was walking in the park when she saw something shiny. She went closer and saw it was a big, round,


In [87]:
# this cell is for tests



### Prompt Engineering
Large language models behave differently based on the prompt, resulting in a new discipline called prompt engineering that focuses on the optimal design of prompts for maximum output. Let's see if we can find some engaging prompts to generate meaningful text.

In [103]:
## ==========================================================
## Play with prompts, try it yourself
prompt = 'Write a joke for me '
max_length = 100
## ==========================================================

if run_check():
    text, _ = text_generation(model,tokenizer,prompt,max_len=max_length)
    print(text)

Write a joke for me 

The little girl thought for a moment and then said, "Why did the chicken cross the road?"

The farmer laughed and said, "I don't know, why?"

The little girl smiled and said, "Because it wanted to get to the other side!"

The farmer was so happy that he gave the little girl a big hug. From then on, the farmer and the little girl were good friends.



## Evaluation of Language Models

Evaluating Natural Language Processing (NLP) models, particularly Large Language Models (LLMs), is essential for understanding their capabilities and limitations. There are two primary approaches to evaluation:

#### 1. Extrinsic Evaluation
This method assesses a model's performance on specific, downstream tasks, providing a practical measure of the model's effectiveness in applied scenarios.

#### 2. Intrinsic Evaluation 
This approach involves identifying inherent properties of a model to estimate its quality, independent of any specific task it was designed to perform. These properties often relate to the model's underlying linguistic capabilities.

---

### Perplexity: A Key Intrinsic Metric

One critical metric for intrinsic evaluation is **perplexity**, which quantifies the uncertainty a language model exhibits during word generation. It reflects how well the model can predict a sequence of text.

#### Calculation of Perplexity

Perplexity is calculated as follows:

1. For each word $x_t$ in the corpus, compute its probability based on all preceding words.
2. Take the inverse of this probability.
3. Calculate the product of these inverse probabilities for all words.
4. Normalize this product by raising it to the power of $\frac{1}{N}$, where $N$ is the total number of words.

Mathematically, this is expressed as:

$$
\text{Perplexity} = \sqrt[N]{\prod_{t=1}^N \frac{1}{P_{LM}(x_t|x_1, \ldots, x_{t-1})}}
$$

Where:
- $N$ is the total number of words in the corpus.
- $P_{LM}(x_t|x_1, \ldots, x_{t-1})$ is the conditional probability of the $t$-th word given all previous words.

> **Note:** To ensure numerical stability and avoid exceptions when the probability $P_{LM}(x_t|x_1, \ldots, x_{t-1})$ is zero, a small epsilon value can be used.

#### Interpretation of Perplexity

A lower perplexity value indicates better performance, as it suggests the model is less uncertain in predicting words.

---

#### Limitations and Considerations

While perplexity is a valuable metric, it  may not always correlate directly with performance on specific downstream tasks. Moreover Perplexity scores are not directly comparable between models with different vocabularies or tokenization schemes.

<div class=" alert alert-warning">
    
## Student Task A5.10

Your task is to implement the function `calculate_perplexity()`, which computes the perplexity based on the output probabilities of the models. 
    
Note: Element t in the input list probabilities is $P_{LM}(x_{t}|x_{1},x_{2}...,x_{t-1})$. and $N=$ len(probabilities)
</div>

In [113]:
# Implementing perplexity calculation
def calculate_perplexity(probabilities,epsilon=1e-12):
    """
    Calculate perplexity given a list of probabilities.
    
    Args:
    probabilities (list): List of probabilities for each token.
    epsilon (float, optional): A small value to prevent division by zero or log of zero.

    
    Returns:
    float: Perplexity value.
    """
    # YOUR CODE HERE
    #raise NotImplementedError()
    perplexity = 1
    for i in range(len(probabilities)):
        if probabilities[i] != 0:            
            perplexity = perplexity * (1 / probabilities[i])
        else:
            perplexity = perplexity * (1 / epsilon)
    return perplexity ** (1 /len(probabilities))

# # Unit Test example:
probs = [0.8, 0.7, 0.1,0.4,0.0]
perplexity = calculate_perplexity(probs)
print(f"Perplexity: {perplexity}") # Output should be Perplexity: 536.970462

Perplexity: 536.9704618928898


In [ ]:
# this cell is for tests


# Zero-Shot Information Retrieval <a class="anchor" id="zero_shot_information_retrieval"></a>

Information Retrieval aims to find and extract information relevant to a user's query. It's widely used in web search, question-answering systems, personal assistants, and chatbots.
In this task, we'll use a pre-trained BERT model (110 million parameters) for zero-shot information retrieval.

>**Note:** "Zero-shot" means the model hasn't been specifically trained for `information retrieval` on this `dataset`.
---

#### Task Overview: 
You are provided with 1,000 short stories and their unique identifiers in `story.csv`. The objective is to retrieve the most relevant story for each query, which consists of 1-2 sentences. The `query.csv` file contains 100 such queries. Your goal is to determine and return the unique identifier of the most relevant story for each query.


#### Solution Strategy: 
We will leverage a language model to generate embeddings—vectorized representations—of both the stories and queries. By calculating the cosine similarity between these embeddings, we will identify and retrieve the story that best matches each query.

In [116]:
# Import the libraries we will be using
import pandas as pd
import tqdm

# 1000 stories with their unique id (story_uid) which is our information database 
story_df = pd.read_csv('story.csv')
story_df.head()

,story_uid,story
0,774,"Once upon a time, there was a little car name..."
1,90,"Once upon a time, there was a white bunny. Th..."
2,961,"Once upon a time, there was a little girl nam..."
3,255,"Once upon a time, there was a little girl who..."
4,382,"One day, a mommy and her little girl went to..."


In [117]:
# We are given query.csv file which contains 100 queries.
# We need to search and return the most similar story (from story_df) corresponding to each query.
query_df = pd.read_csv('query.csv')
query_list = list(query_df['query'])
query_list[:2]

[' "Give that back!" Beth said. Her brother ignored her and tried to run away with it, but Beth was determined not to let him take it',
 '" From that day on, Lily played with her toy ambulance and pretended to be a smart doctor who helped people in need. She knew that one day, she would be just like the real ambulance and help even more people']

### Extracting Embedding Vectors Using a Pre-Trained LLM Model

There are various methods to extract embeddings from a pre-trained language model, each suited for different use cases. The output from the last layer of the language model typically has the shape `(B, T, d)`, where:
- `B` is the batch size,
- `T` is the sequence length (number of tokens), and
- `d` is the feature dimension in each token's representation.

Assuming `output_tensor` (a NumPy/ PyTorch tensor) is the output from the last layer with shape `(B, T, d)`, where `B=1`, the following are some common approaches to extract embeddings:

1. **Last Token Embedding:** Extracts the embedding of the last token in the sequence: `last_token_embedding = output_tensor.squeeze(0)[-1, :]`.
2. **[CLS] Token Embedding:** Uses the embedding of the [CLS] token, which is often located at the first position and represents the entire sequence (specific to BERT models): `cls_token_embedding = output_tensor.squeeze(0)[0, :]`
3. **Mean Pooling:** Computes the mean of all token embeddings, providing a generalized representation of the sequence: `mean_embedding = output_tensor.mean(dim=1).squeeze(0)`
4. **Weighted Mean Pooling:** Applies a weighted mean where the influence of each token's embedding decreases linearly from the last token to the first. This can be implemented using the `get_weighted_mean_emb()` function : `weighted_mean_embedding = get_weighted_mean_emb(last_layer_embs)`

For our model, we found that Mean Pooling and especially Weighted Mean Pooling yield the best results in terms of capturing the sequence's semantic information.

> **Note:** For models of the `AutoModelForCausalLM` type, the `model.transformer()` method is used to obtain the output from the last feature layer.


In [118]:
import torch.nn.functional as F


# Function to extract mean weighted embeddings.
def get_weighted_mean_emb(last_layer_embs):
    phrase_embedding_mean_weighted = torch.zeros(last_layer_embs.size(-1))
    len_s = last_layer_embs.size(0)
    for j, j_token in enumerate(last_layer_embs):
        phrase_embedding_mean_weighted += ((j / len_s) * j_token)
    normalized = F.normalize(phrase_embedding_mean_weighted, p=2, dim=-1)
    return normalized.detach().numpy()


# Here we have precomputed the embeddings for each story to save some time.
load_precomputed_embeddings = True

if not load_precomputed_embeddings and run_check():
    story_emb_list = []
    for story in tqdm.tqdm(story_df['story']):
        # Mean
        #story_emb_list.append(model.transformer(tokenizer.encode(story, return_tensors="pt"))[0].mean(dim=1).squeeze(0).detach().numpy())
        
        # Weighted mean, using the get_weighted_mean_emb() function
        story_emb_list.append(get_weighted_mean_emb(model.transformer(tokenizer.encode(story, return_tensors="pt"))[0].squeeze(0)))
        
    story_emb_array = np.stack(story_emb_list)
else:
    story_emb_array = np.load('story_embeddings.npy')

uids = np.array(list(story_df['story_uid']))

In [119]:
# We also want to get the embeddings of each query.
query_emb_list = []
for query in query_list:
    query_emb_list.append(get_weighted_mean_emb(model.transformer(tokenizer.encode(query, return_tensors="pt"))[0].squeeze(0)))
query_emb_array = np.stack(query_emb_list)

## Visualization of Embeddings

Each stories and queries are embedded to a 768 dimensional vector space where their position and distance represents their semantic similarities. For visualization purposes, those vectors are projected down to 2 dimension using tSNE method (PCA is also an alternatives) as shown below. 

In [120]:
## The code to generate TSNE plot
#from sklearn.manifold import TSNE
#import matplotlib.pyplot as plt
#story_embedded = TSNE(n_components=2, init='random', perplexity=3).fit_transform(story_emb)
#plt.scatter(story_embedded[:,0],story_embedded[:,1],marker='o')
#query_embedded = TSNE(n_components=2, init='random', perplexity=3).fit_transform(query_emb[:30])
#plt.scatter(query_embedded[:,0],query_embedded[:,1],marker='+')
#plt.legend(["story embeddings" , "Query embeddings"])

<img src="tsne.png" alt="drawing" />

The information retrieval problem has been simplified to a search problem in a vector space where both the data and the query reside. To determine the closest data point for each query, we will utilize the cosine similarity measure.

### Cosine Similarity

Cosine similarity is a measure of similarity between two non-zero vectors in an n-dimensional space. It quantifies the cosine of the angle between the vectors, providing an indication of their orientation rather than their magnitude. This metric is widely used to assess the similarity between documents or feature vectors.

Given two vectors of attributes $\mathbf{a}$ and $\mathbf{b}$, the cosine similarity $\cos(\theta)$ is defined as:

$$
\cos(\theta) = \frac{\mathbf{a} \cdot \mathbf{b}}{\|\mathbf{a}\| \|\mathbf{b}\|}
$$

Where:
- $\mathbf{a} \cdot \mathbf{b}$ represents the dot product of the vectors.
- $\|\mathbf{a}\|$ and $\|\mathbf{b}\| $ are the magnitudes (or Euclidean norms) of the vectors $\mathbf{a}$ and $ \mathbf{b}$, respectively.

The magnitude of a vector $\mathbf{a}$ is calculated as:

$$
\|\mathbf{a}\| = \sqrt{a_1^2 + a_2^2 + \cdots + a_n^2}
$$


Cosine similarity ranges from -1 to 1:
- A value of 1 indicates that the vectors are identical in direction.
- A value of 0 indicates that the vectors are orthogonal (i.e., completely dissimilar in terms of direction).
- A value of -1 indicates that the vectors are diametrically opposed.

> Note: In Python, the magnitude of a vector can be computed using the `np.linalg.norm(vector)` function from the NumPy library.


<div class=" alert alert-warning">
    
## Student Task A5.11

Your task is to implement the function `cosine_similarity()`, which computes the similarity between two vectors vector1 and vector2. 
</div>

In [126]:
# implement cosine similarity measure between two vectors.
def cosine_similarity(vector1, vector2):
    '''
    Calculate Cosine similarity between two vectors.
    
    Parameters:
    vector1: numpy array of shape (N,)
    vector2: numpy array of shape (N,)
    
    Returns:
    similarity: cosine similarity value.
    
    '''
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    similarity = np.dot(vector1, vector2) / (norm1 * norm2)
    
    
    return similarity


# Unit test case
a = np.array([1.0, 4.1, 5.6])
b = np.array([0.1, -1.5, 3.2])
sim_score = cosine_similarity(a, b)  # sim_score = 0.47879
print(sim_score)

0.4787905921770076


In [127]:
# this cell is for tests


### Example of document retrieval for a single query

Feel free to change the `query` string in the below cell and see what story is retrieved.

In [128]:
## ==========================================================
## A single query. Feel free to try your own query!
#query = 'picking berries'
#query = 'loud sound'
query = 'the dog'
## ==========================================================

if run_check():
    print(f'Query: "{query}"')
    
    # Embedding of the query
    #query_embed = model.transformer(tokenizer.encode(query, return_tensors="pt"))[0].mean(dim=1).squeeze(0).detach().numpy()
    #query_embed = model.transformer(tokenizer.encode(query, return_tensors="pt"))[0].squeeze(0)[-1].detach().numpy()
    query_emb = get_weighted_mean_emb(model.transformer(tokenizer.encode(query, return_tensors="pt"))[0].squeeze(0))

    # Cosine similarity of the query with all the documnts/stories in our data
    similarity_list = [cosine_similarity(query_emb, x) for x in story_emb_array]

    # Index of the retrived document and its similarity score to the query.
    index = similarity_list.index(max(similarity_list))
    score = similarity_list[index]
    story = story_df.iloc[index]["story"]
    uid = story_df.iloc[index]['story_uid']
    
    print(f'\nRetrieved story: {story_df.iloc[index]["story"]}')
    print(f'\nUnique id of retreived document: {uid}')

Query: "the dog"

Retrieved story:   Once upon a time, there was a rare puppy. The puppy was very cute. One day, the puppy fell asleep in the sun. The sun was nice and warm. It felt so good that the puppy started to change. His fur grew brighter and brighter. "Wow!" said a girl. She was walking by. "He looks so different. He was grey before. Now he is white and fluffy!" The puppy opened his eyes and said, "I changed!" He jumped up and started to bark. The girl laughed. "You sure did change! You are the most rare and beautiful puppy I have ever seen." The puppy smiled and started to lick the girl's face. He was so happy that he had changed. 

Unique id of retreived document: 737


<div class=" alert alert-warning">
    
## Student Task A5.12

Your task is to implement the function `predict_uid()`, which yields the predicted document uids (predicted_uid_list). The instructions for a single example are already shown in the cell above.
</div>

In [145]:
def predict_uid(query_embs, story_embs, story_df):
    '''
    Predict the uid for each query in the query_emb array.
    
    Args:
        query_emb (np.array): Array of query embeddings.
        story_emb (np.array): Array of story embeddings.
        story_df (pd.DataFrame): DataFrame containing story information including 'story_uid'.
    Return:
        predicted_uid_list (List): List of predicted story uids corresponding to the queries.
    
    '''
    #list to append predicted story/document uids corresponding to the queries.  
    predicted_uid_list = []
    
    # YOUR CODE HERE
    #raise NotImplementedError()
    for query_emb in query_embs:
        similarity_list = [cosine_similarity(query_emb, x) for x in story_embs]
        index = similarity_list.index(max(similarity_list))
        uid = story_df.iloc[index]['story_uid']
        predicted_uid_list.append(uid)
        
    return predicted_uid_list

# #unit test example:
predicted_uids_test = predict_uid(query_emb_array[:2],story_emb_array,story_df) # correct output should be predicted_uids_test = [957, 245]
print(predicted_uids_test)

[957, 245]


In [146]:
# this cell is for tests

